Note that the labelling of these sections is based on the version of the paper completed on 15/08/22. This cell should be updated in the event of a change to the labelling in the paper.

Currently running on Sagemath version 9.4.rc0, Release Date: 2021-07-27.
Using Python 3.9.5.

This notebook is intended to be ran sequentially from the beginning. Changing the order in which cells are ran may lead to errors. 

Note in the following we will often take a value being less than $10^{-10}$ to be sufficient to consider it as zero. This is arbitrary and not connected to our choice of precision. If one wished to be precise an investigation of how any error decreases as precision is increased would be appropriate. 

## Theorem 2.12

In [1]:
# We will first want to verify the claims about Weber's period matrix. 
# To do this we will go via the Riemann matrix calculated by Sage. 
from sage.schemes.riemann_surfaces.riemann_surface import numerical_inverse, integer_matrix_relations
from sage.schemes.riemann_surfaces.riemann_surface import RiemannSurface

# Let's define the curve
R.<x, y> = QQ[]
f = x*(y^5+1) + (x*y)^2 - x^4*y - 2*y^3
integration_method = 'rigorous'
prec = 60
S = RiemannSurface(f, prec=prec, integration_method=integration_method)
g = S.genus

# To relate this to the R&R model we will need to calculate tau_0 := t0 
# to the desired precision. We will do this using the hypergeometric function
# to invert. 
j0 = -29^3*5*2^(-5)
alp = S._CCz.gen()
roots = (4*j0*alp*(1-alp)-1728).roots(multiplicities=False)
# Note one could use the Arb's hypergeometric to get rigorous bounds on these, 
# but that won't be necessary for us
t1, t2 = (I*hypergeometric([1/6,5/6],[1],1-al).hypergeometric_simplify()/\
          hypergeometric([1/6,5/6],[1],al).hypergeometric_simplify() 
          for al in roots)

# We need to choose the right root, which for us will have real part 0.5. 
if (t1.real_part()-1/2).abs()>1e-10:
    t1 = t2
if (t1.real_part()-1/2).abs()>1e-10:
    raise ValueError("Invalid taus found, check calculation.")
    
# We will then map this to the desired form of t0 using a Mobius transform
from sage.geometry.hyperbolic_space.hyperbolic_model import moebius_transform
t0 = moebius_transform(matrix(2,2,[-1,1,-2,1]), S._CC(t1))
# Check it is valid
if (elliptic_j(t0)+29^3*5/2^5).norm()+(elliptic_j(5*t0)+25/2).norm()>1e-10:
    raise ValueError("Value of t0 found doesn't satisfy the desired conditions.")
# Here we will now print t0 to be concrete about which value we have taken. 
print("t0:", t0, "\n")
# This let's us define the R&R Riemann matrix. 
RM_Braden = t0*matrix([[4,1,-1,1],[1,4,1,-1],[-1,1,4,1],[1,-1,1,4]])

# We now want to define Weber's period matrix wrt to a canonical homology basis. 
# We write down the intersection matrxi Weber gives. 
Int_matrix = matrix([[0,1,1,-1,-1,0,0,1],
                    [-1,0,1,1,0,1,0,-1],
                    [-1,-1,0,1,0,-1,0,0],
                    [1,-1,-1,0,0,0,1,0],
                    [1,0,0,0,0,1,1,-1],
                    [0,-1,1,0,-1,0,1,1],
                    [0,0,0,-1,-1,-1,0,1],
                    [-1,1,0,0,1,-1,-1,0]])

# We canonicalise it. 
from sage.matrix.symplectic_basis import symplectic_basis_over_ZZ
C = symplectic_basis_over_ZZ(Int_matrix)[1].transpose()

# We construct Weber's period matrix.
xi = exp(S._RR.pi()*I/5)
Phi = (S._RR(sqrt(5))+1)/2
# To get l we invert the final expression in Corollary 5.3 of Weber, using the fact that 
# we can minimise to find the approximation to a reliable accuracy. 
def loss_fn(t):
    tau = CC(t[0]+I*t[1])
    j1 = elliptic_j(tau)
    return (j1+25/2).abs()^2
minxy = minimize(loss_fn, [CC(0.554051), CC(0.832482)], xtol=1e-60)
trat = CC(minxy[0]+minxy[1]*I)

l = (xi*xi.conjugate()*(1-xi)-
     trat*xi.conjugate()*(1-xi.conjugate()))/(xi.conjugate()-trat)
PM_list = [[xi^(2*k)*(1-xi^4), xi^(4*k+7)*(1-xi^8)*(l*Phi-1),
           xi^(8*k+6)*(1-xi^6)*Phi*(1-l), xi^(6*k+4)*(1-xi^2)*l] 
          for k in range(4)]
PM_list += [[xi^(4*k+7)*(1-xi^8)*(l*Phi-1), 
             xi^(8*k+6)*(1-xi^6)*Phi*(1-l),
             xi^(6*k+4)*(1-xi^2)*l, xi^(2*k)*(1-xi^4)] 
          for k in range(4)]
PM_W = Matrix(8, 4, PM_list).transpose()
PM_W = PM_W*C
A_W = PM_W[:,0:g]
B_W = PM_W[:,g:]
RM_W = numerical_inverse(A_W)*B_W

# We now use code written by NB to get the symplectic automorphism group to find the relations
# between the two 
Rs = integer_matrix_relations(RM_W, RM_Braden)
r = len(Rs)
A = PolynomialRing(QQ, r, 'x')
gensA = A.gens()
        # Use that the trace is positive definite; we could also put this as an
        # extra condition when determining the endomorphism basis to speed up
        # that calculation slightly
R = sum(gensA[i] * Rs[i].change_ring(A) for i in range(r))
tr = (R * S.rosati_involution(R)).trace()
        # Condition tr = 2 g creates ellipsoid
M = Matrix(ZZ, r, r, [tr.derivative(gen1).derivative(gen2)
                        for gen1 in gensA for gen2 in gensA])

vs = M.__pari__().qfminim(4*g)[2].sage().transpose()

vs = [v for v in vs if v * M * v == 4*g]
vs += [-v for v in vs]
RsIso = []
for v in vs:
    R = sum(v[i] * Rs[i] for i in range(r))
    if R * S.rosati_involution(R) == 1:
        RsIso.append(R)

if len(RsIso)!=240:
    raise ValueError("Period matrices do not give isomorphic Jacobians, error present.")
    
# We pick out an element which gives an isomorphism with few non-zero entries.
# This may not be the exact matrix in the paper as the precision used will affect which matrix 
# is printed. 
non_zeros = [sum([bool(ri) for ri in RIs.list()]) for RIs in RsIso]
min_ind = non_zeros.index(min(non_zeros))
R0 = RsIso[min_ind]
print(R0,"\n")
D0 = R0[0:g, 0:g]
B0 = R0[0:g, g:]
C0 = R0[g:, 0:g]
A0 = R0[g:, g:]
if (max([el.norm() for el in (B0+RM_W*A0-(D0+RM_W*C0)*RM_Braden).list()]) > 1e-10):
    raise ValueError("Error in transform. ")
    
R_paper = matrix([[0, 0, 0, -1, 1, -1, 0, 2],
                 [0, -1, 1, -1, 1, 1, -1, 1],
                 [1, 0, 1, -1, -1, -1, -1, 1],
                 [1, 0, 0, 0, -2, 0, 0, -1],
                 [0, 1, 0, 0, 0, -2, -1, 0],
                 [1, 0, 1, 0, -2, -1, -1, -1],
                 [0, -1, 0, -1, 1, 2, 1, 1],
                 [1, 0, 1, 0, -1, -1, -2, -1]])


t0: 0.50000000000000000 + 0.18667646246280991*I 

[ 0  0  0  1  0  0  1  2]
[ 0 -1  1  0 -1 -2  1  1]
[ 1  0  1  0  2  1  1  1]
[ 0  1  0  0  1  2  0  0]
[ 1  0  0  0  2  1  0  1]
[ 1  0  1 -1  1  1  1 -1]
[-1  0  0  1 -1 -1  1  2]
[ 1  0  1 -1  1  2  1 -1] 



In [2]:
# To show that the R&R period matrix is valid straight from Sage, we can do the above but
# find a transform from the numerically calculated Riemann matrix. 
RM = S.riemann_matrix()

Rs = integer_matrix_relations(RM, RM_Braden)
r = len(Rs)
A = PolynomialRing(QQ, r, 'x')
gensA = A.gens()
        # Use that the trace is positive definite; we could also put this as an
        # extra condition when determining the endomorphism basis to speed up
        # that calculation slightly
R = sum(gensA[i] * Rs[i].change_ring(A) for i in range(r))
tr = (R * S.rosati_involution(R)).trace()
        # Condition tr = 2 g creates ellipsoid
M = Matrix(ZZ, r, r, [tr.derivative(gen1).derivative(gen2)
                        for gen1 in gensA for gen2 in gensA])

vs = M.__pari__().qfminim(4*g)[2].sage().transpose()

vs = [v for v in vs if v * M * v == 4*g]
vs += [-v for v in vs]
RsIso = []
for v in vs:
    R = sum(v[i] * Rs[i] for i in range(r))
    if R * S.rosati_involution(R) == 1:
        RsIso.append(R)

print(len(RsIso)==240)

True
